In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

import jax
from jax import numpy as jnp
from jax.config import config
from flax.training import checkpoints
import flax.linen as nn

import nest_asyncio
nest_asyncio.apply()

from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import NearestNDInterpolator
from scipy.interpolate import interp1d

from scipy.optimize import minimize
from scipy.stats import qmc
# Constants
from scipy.constants import Boltzmann, Avogadro
kb = Boltzmann # [J/K] Boltzman's constant
Na = Avogadro  # [mol-1] Avogadro's Number
R = Na * kb    # [J mol-1 K-1] Ideal gas constant

sys.path.append("../../")
from python_helpers.feanneos import HelmholtzModel
from python_helpers.feanneos import helper_solver_funs, helper_jitted_funs
from python_helpers.transport_properties import TransportModel_PVT_Tinv
from python_helpers import helper_get_alpha
from python_helpers import linear_activation
from python_helpers.data_figures import mie_params_of_vle_sle_sve as mie_params_of


PRECISSION = 'float64'
if PRECISSION == 'float64':
    config.update("jax_enable_x64", True)
    type_np = np.float64
    type_jax = jnp.float64
else:
    config.update("jax_enable_x32", True)
    type_np = np.float32
    type_jax = jnp.float32

np.seterr(all="ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
#######################################
# type and optimization configuration #
#######################################

of_type = "vle_sle_sve"
# ranges for sigma, eps, lambda_r
l_bounds = [2.5, 100., 9.]
u_bounds = [3.5, 300., 26.]
m_base = 5
params_file = f'optimized_mie_params_{of_type}.csv'

In [3]:
###########

filename = '../../computed_files/phase_equilibria_solid.xlsx'
excel_file = pd.ExcelFile(filename)

df_info = pd.read_excel(excel_file, sheet_name='info')
df_vle = pd.read_excel(excel_file, sheet_name='vle')
df_sle = pd.read_excel(excel_file, sheet_name='sle')
df_sve = pd.read_excel(excel_file, sheet_name='sve')

# critical point information interpolation
input_crit_interp = df_info['alpha'].to_numpy()
output_crit_interp = df_info[['rhocad_model', 'Tcad_model', 'Pcad_model']].to_numpy()
crit_interp = interp1d(input_crit_interp, output_crit_interp.T, fill_value='extrapolate')

# triple point information interpolation
input_triple_interp = df_info['alpha'].to_numpy()
output_triple_interp = df_info[['rhovad_triple', 'rholad_triple', 'rhosad_triple',
                                'T_triple', 'P_triple']].to_numpy()
triple_interp = interp1d(input_triple_interp, output_triple_interp.T, fill_value='extrapolate')

# Interpolating VLE
input_vle_interp = df_vle[['alpha', 'Tr_vle_model']].to_numpy()
output_vle_interp = df_vle[['P_vle_model', 'rhov_vle_model', 'rhol_vle_model']].to_numpy()
vle_interp = LinearNDInterpolator(input_vle_interp, output_vle_interp)

# Interpolating SLE
input_sle_interp = df_sle[['alpha', 'T_sle_model']].to_numpy()
output_sle_interp = df_sle[['P_sle_model', 'rhol_sle_model', 'rhos_sle_model']].to_numpy()
sle_interp = NearestNDInterpolator(input_sle_interp, output_sle_interp)

# Interpolating SVE
input_sve_interp = df_sve[['alpha', 'T_sve_model']].to_numpy()
output_sve_interp = df_sve[['P_sve_model', 'rhov_sve_model', 'rhos_sve_model']].to_numpy()
sve_interp = NearestNDInterpolator(input_sve_interp, output_sve_interp)

alphas_sle = np.unique(df_sle['alpha'])
Tsle_max = np.zeros_like(alphas_sle)
for i, alpha in enumerate(alphas_sle):
    Tsle_max[i] = df_sle[df_sle['alpha'] == alpha]['T_sle_model'].max()
sle_maxT_interp = interp1d(alphas_sle, Tsle_max, fill_value='extrapolate')

interpd_dict = {'crit_interp': crit_interp, 'vle_interp': vle_interp, 
                'sle_interp': sle_interp, 'sve_interp': sve_interp, 
                'triple_interp': triple_interp, 'sle_maxT_interp': sle_maxT_interp}

In [4]:
#########################
# Loading FE-ANN(s) EoS #
#########################

ckpt_folder = '../../ann_models/feanns_eos'
prefix_params = 'FE-ANN-EoS-params_'
###
Tscale = 'Tinv'
seed = 17
factor = 0.01
EPOCHS = 50000
traind_model_folder = f'models_{Tscale}_factor{factor:.2f}_seed{seed}'
ckpt_folder_model = os.path.join(ckpt_folder, traind_model_folder)
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder_model, target=None, prefix=prefix_params, step=EPOCHS)
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}

fun_dic_solid = helper_jitted_funs(helmholtz_model, helmholtz_params)


In [5]:
# Reading data from NIST
filename = 'argon.xlsx'
DataFile = pd.ExcelFile(filename)

if not os.path.exists(params_file):
    sampler = qmc.Sobol(d=3, scramble=False)
    sample = sampler.random_base2(m=m_base)

    scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

    sigma_guess = scaled_sample[:, 0]
    epsilon_guess = scaled_sample[:, 1]
    lr_guess = scaled_sample[:, 2]

    n = len(sigma_guess)

    index = np.arange(n)
    solved = np.zeros(n, dtype=bool)
    sigma_sol = np.nan * np.ones(n)
    epsilon_sol = np.nan * np.ones(n)
    lr_sol = np.nan * np.ones(n)
    of = np.nan * np.ones(n)
    solved_success = np.zeros(n, dtype=bool)

    df = pd.DataFrame({'index': index, 
                    'sigma_guess': sigma_guess, 'epsilon_guess': epsilon_guess, 'lr_guess': lr_guess,
                    'optimized': solved, 
                    'sigma_sol': sigma_sol, 'epsilon_sol': epsilon_sol, 'lr_sol': lr_sol, 'of': of, 'solved_success': solved_success})
    
    df.to_csv(params_file, index=False)


In [6]:
# bounds to control sigma, epsilon and lr
bounds = ((None, None), (None, None), (7., 34.))

kwargs = {'DataFile': DataFile,
          'fun_dic': fun_dic_solid,
          'interpd_dict': interpd_dict,
          'lambda_a': 6.,
          'weight_rhov_vle':0.0,
          'weight_sle': 1e-2,
          'weight_sve': 1.,
          'weight_enthalpy': 1.0,
          'add_critical_point': True,    
          'add_triple_point': True,
          }
args = tuple(kwargs.values())

In [7]:
df = pd.read_csv(params_file)
n = len(df)
for i in range(n):
    if not df.loc[i, 'optimized']:
        inc0 = df.loc[i, ['sigma_guess', 'epsilon_guess', 'lr_guess']].to_list()
        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds, options={'maxiter':10})

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)
    print(i, df.loc[i, 'optimized'], df.loc[i, 'of'])

0 True 2.086976537964751
1 True 0.7740150323233339
2 True 0.0021102955002594
3 True nan
4 True 1.3003251306533186
5 True nan
6 True 0.7460458676398152
7 True nan
8 True 0.1514127966262444
9 True nan
10 True 4.418258445789139
11 True 0.0021102954998239
12 True 0.0021102954997657
13 True nan
14 True 0.256139715279777
15 True nan
16 True 0.5212559885794518
17 True nan
18 True 0.0021102954982296
19 True nan
20 True 0.4819475567194087
21 True nan
22 True 0.7938194638103291
23 True nan
24 True 0.0021102954996922
25 True nan
26 True 1.2845643710389198
27 True nan
28 True 0.3248149890654646
29 True nan
30 True 0.1223632223255363
31 True 1.5436424823496369


In [8]:
df = pd.read_csv(params_file)
n = len(df)

of_order = np.argsort(df['of'].to_numpy())

for i in of_order[:5]:
    if not df.loc[i, 'solved_success']:
        inc0 = df.loc[i, ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()

        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds)

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)


In [9]:
of_order = np.argsort(df['of'].to_numpy())
df.loc[of_order].head(n=10)

,index,sigma_guess,epsilon_guess,lr_guess,optimized,sigma_sol,epsilon_sol,lr_sol,of,solved_success
18,18,3.34375,143.75,12.71875,True,3.421741,124.622107,13.527322,0.002110,True
24,24,2.65625,131.25,18.03125,True,3.421739,124.622101,13.527317,0.002110,True
12,12,2.81250,137.50,14.31250,True,3.421739,124.622055,13.527304,0.002110,True
11,11,2.93750,212.50,12.18750,True,3.421741,124.622068,13.527316,0.002110,True
2,2,3.25000,150.00,13.25000,True,3.421740,124.622119,13.527321,0.002110,True
30,30,3.03125,106.25,15.90625,True,3.143830,123.870885,12.654633,0.122363,False
8,8,2.68750,162.50,24.93750,True,3.689322,132.373971,15.848483,0.151413,False
14,14,3.06250,187.50,10.06250,True,3.266667,125.000000,12.075000,0.256140,False
28,28,2.78125,156.25,11.65625,True,3.208166,121.752829,12.171907,0.324815,False
20,20,2.96875,118.75,23.34375,True,3.364583,140.520833,17.118750,0.481948,False


In [10]:
param_best = df.loc[of_order[0], ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()
param_best

[3.421740778753814, 124.62210662519855, 13.527322089446582]